In [1]:
import datetime
import backtrader as bt
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# **PASO 3 ESTRATEGIAS DE TRADING AUTOMATIZADO**

In [2]:
import yfinance as yf
import pandas as pd

# Definir los tickers
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 
           'TSLA', 'BRK-B', 'META', 'V', 'JNJ', 
           'XOM', 'UNH', 'PG', 'JPM', 'MA']

# Descargar los datos históricos
datos = yf.download(tickers, start='2015-03-03', end='2025-03-03')

# Extraer solo los precios de cierre
datos = datos['Close']

# Asegurarse de que la fecha sea el índice
datos.reset_index(inplace=True)
datos.set_index('Date', inplace=True)

# Mostrar las primeras filas
print(datos.head())

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  15 of 15 completed

Ticker           AAPL       AMZN       BRK-B      GOOGL        JNJ        JPM  \
Date                                                                            
2015-03-03  28.889668  19.230499  145.929993  28.834970  77.579605  47.069855   
2015-03-04  28.706539  19.136000  144.919998  28.812054  77.056557  47.191391   
2015-03-05  28.230854  19.391500  146.059998  28.966496  77.716072  47.092655   
2015-03-06  28.273285  19.004499  145.279999  28.541536  75.889153  46.249542   
2015-03-09  28.393885  18.927999  146.130005  28.601318  76.306099  46.712849   

Ticker             MA       META       MSFT      NVDA         PG       TSLA  \
Date                                                                          
2015-03-03  85.579048  79.297081  37.353813  0.534396  64.354591  13.304000   
2015-03-04  86.275253  80.592140  37.163948  0.540419  63.742466  13.496000   
2015-03-05  87.310036  80.900955  37.207104  0.550779  63.954060  13.375333   
2015-03-06  85.409760  79.705528  36.

### **MEDIAS MÓVILES**

In [4]:
# Estrategia Media Móvil Corregida
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Configuración de estilo
COLOR_COMPRA = '#4B9B44'
COLOR_VENTA = '#DE4F36'

def generar_senales_media_movil(precios, ventana_corta=20, ventana_larga=100):
    """Genera señales de compra/venta usando cruce de medias móviles"""
    senales = pd.DataFrame(index=precios.index)
    senales['precio'] = precios
    senales['media_corta'] = precios.rolling(ventana_corta).mean()
    senales['media_larga'] = precios.rolling(ventana_larga).mean()
    
    # Detectar cruces
    senales['cruce'] = np.where(senales['media_corta'] > senales['media_larga'], 1, 0)
    senales['senal'] = senales['cruce'].diff()
    return senales

def ejecutar_operaciones(precios, senales, capital_inicial=5000):
    """Ejecuta las operaciones basadas en las señales"""
    capital = capital_inicial
    acciones = 0
    operaciones = []
    historial = []
    
    for fecha in precios.index:
        precio_actual = precios[fecha]
        senal = senales.loc[fecha, 'senal']
        
        if senal == 1:  # Señal de compra
            if capital > 0:
                acciones = capital / precio_actual
                capital = 0
                operaciones.append(('compra', fecha, precio_actual))
                
        elif senal == -1:  # Señal de venta
            if acciones > 0:
                capital = acciones * precio_actual
                acciones = 0
                operaciones.append(('venta', fecha, precio_actual))
                
        # Registrar valor del portafolio
        valor_actual = capital + (acciones * precio_actual)
        historial.append(valor_actual)
    
    return historial, operaciones

def graficar_resultados(activo, precios, senales, operaciones):
    """Genera gráfico completo con indicadores y operaciones"""
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), sharex=True, gridspec_kw={'height_ratios': [3, 1]})
    
    # Gráfico de precios
    ax1.plot(precios, label='Precio', color='#1f77b4', alpha=0.8)
    ax1.plot(senales['media_corta'], label=f'MA {ventana_corta}', linestyle='--', color='#ff7f0e')
    ax1.plot(senales['media_larga'], label=f'MA {ventana_larga}', linestyle='--', color='#2ca02c')
    
    # Marcadores de operaciones
    compras = [op for op in operaciones if op[0] == 'compra']
    ventas = [op for op in operaciones if op[0] == 'venta']
    
    ax1.scatter(
        x=[op[1] for op in compras],
        y=[op[2] for op in compras],
        marker='^', color=COLOR_COMPRA, s=100,
        edgecolors='black', label='Compra', zorder=3
    )
    
    ax1.scatter(
        x=[op[1] for op in ventas],
        y=[op[2] for op in ventas],
        marker='v', color=COLOR_VENTA, s=100,
        edgecolors='black', label='Venta', zorder=3
    )
    
    ax1.set_title(f'Estrategia Medias Móviles - {activo}', fontsize=14, pad=20)
    ax1.set_ylabel('Precio (USD)')
    ax1.legend(loc='upper left')
    ax1.grid(True, alpha=0.4)
    
    # Gráfico de señales
    ax2.plot(senales['cruce'], label='Posición', color='#9467bd', alpha=0.7)
    ax2.set_yticks([0, 1])
    ax2.set_yticklabels(['Venta', 'Compra'])
    ax2.set_ylabel('Señal')
    ax2.grid(True, alpha=0.4)
    
    # Formato de fechas
    locator = mdates.AutoDateLocator()
    formatter = mdates.ConciseDateFormatter(locator)
    ax1.xaxis.set_major_locator(locator)
    ax1.xaxis.set_major_formatter(formatter)
    
    plt.tight_layout()
    return fig

# Parámetros
ventana_corta = 20
ventana_larga = 100
capital_inicial = 5000

# Procesamiento para cada activo
resultados = []
for activo in datos.columns:
    precios = datos[activo]
    
    # Generar señales
    senales = generar_senales_media_movil(
        precios, 
        ventana_corta=ventana_corta,
        ventana_larga=ventana_larga
    )
    
    # Ejecutar operaciones
    historial, operaciones = ejecutar_operaciones(precios, senales, capital_inicial)
    
    # Calcular métricas
    rentabilidad_bruta = historial[-1] - capital_inicial
    rentabilidad_porcentual = (rentabilidad_bruta / capital_inicial) * 100
    
    # Almacenar resultados
    resultados.append({
        'Activo': activo,
        'Capital Final': round(historial[-1], 2),
        'Rentabilidad Bruta': round(rentabilidad_bruta, 2),
        'Rentabilidad %': round(rentabilidad_porcentual, 2),
        'Operaciones': len(operaciones)
    })
    
    # Generar gráfico
    fig = graficar_resultados(activo, precios, senales, operaciones)
    plt.savefig(f'media_movil_{activo}.png')  # Guardar imagen
    plt.close()  # Cerrar figura para liberar memoria

# Mostrar resultados
resultados_df_m = pd.DataFrame(resultados)
print("\nResultados por Activo:")
print(resultados_df_m.to_string(index=False))


Resultados por Activo:
Activo  Capital Final  Rentabilidad Bruta  Rentabilidad %  Operaciones
  AAPL       17218.77            12218.77          244.38           33
  AMZN       20577.15            15577.15          311.54           37
 BRK-B       11363.31             6363.31          127.27           31
 GOOGL       14245.96             9245.96          184.92           29
   JNJ        5340.90              340.90            6.82           29
   JPM       17775.45            12775.45          255.51           29
    MA       11539.06             6539.06          130.78           33
  META       39081.25            34081.25          681.63           21
  MSFT       23995.04            18995.04          379.90           22
  NVDA      404209.28           399209.28         7984.19           28
    PG        7940.45             2940.45           58.81           31
  TSLA       27662.38            22662.38          453.25           35
   UNH        7777.21             2777.21           5

# **BOLLINGER**

In [36]:
# bollinger_completo.py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Configuración de estilo
COLOR_COMPRA = '#4B9B44'
COLOR_VENTA = '#DE4F36'

def generar_senales_bollinger(precios, ventana=20, desviaciones=2):
    """Genera señales de trading usando Bollinger Bands"""
    senales = pd.DataFrame(index=precios.index)
    senales['precio'] = precios
    senales['sma'] = precios.rolling(ventana).mean()
    senales['std'] = precios.rolling(ventana).std()
    
    # Calcular bandas
    senales['upper'] = senales['sma'] + (desviaciones * senales['std'])
    senales['lower'] = senales['sma'] - (desviaciones * senales['std'])
    
    # Generar señales
    senales['senal'] = 0
    senales['senal'] = np.where(senales['precio'] < senales['lower'], 1, senales['senal'])
    senales['senal'] = np.where(senales['precio'] > senales['upper'], -1, senales['senal'])
    
    return senales

def ejecutar_operaciones(precios, senales, capital_inicial=5000):
    """Ejecuta las operaciones basadas en las señales"""
    capital = capital_inicial
    acciones = 0
    operaciones = []
    historial = []
    
    for fecha in precios.index:
        precio_actual = precios[fecha]
        senal = senales.loc[fecha, 'senal']
        
        if senal == 1 and capital > 0:  # Compra
            acciones = capital / precio_actual
            capital = 0
            operaciones.append(('compra', fecha, precio_actual))
            
        elif senal == -1 and acciones > 0:  # Venta
            capital = acciones * precio_actual
            acciones = 0
            operaciones.append(('venta', fecha, precio_actual))
            
        # Registrar valor del portafolio
        valor_actual = capital + (acciones * precio_actual)
        historial.append(valor_actual)
    
    return historial, operaciones

def graficar_bollinger(activo, precios, senales, operaciones):
    """Genera gráfico completo con Bollinger Bands y operaciones"""
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), sharex=True, gridspec_kw={'height_ratios': [3, 1]})
    
    # Gráfico de precios
    ax1.plot(precios, label='Precio', color='#1f77b4', alpha=0.8, linewidth=1.5)
    ax1.plot(senales['sma'], label=f'SMA {ventana}', color='#ff7f0e', linestyle='--')
    ax1.plot(senales['upper'], label='Banda Superior', color='#2ca02c', linestyle=':', linewidth=1.2)
    ax1.plot(senales['lower'], label='Banda Inferior', color='#d62728', linestyle=':', linewidth=1.2)
    ax1.fill_between(precios.index, senales['upper'], senales['lower'], alpha=0.1, color='gray')
    
    # Marcadores de operaciones
    compras = [op for op in operaciones if op[0] == 'compra']
    ventas = [op for op in operaciones if op[0] == 'venta']
    
    ax1.scatter(
        x=[op[1] for op in compras],
        y=[op[2] for op in compras],
        marker='^', color=COLOR_COMPRA, s=100,
        edgecolors='black', label='Compra', zorder=3
    )
    
    ax1.scatter(
        x=[op[1] for op in ventas],
        y=[op[2] for op in ventas],
        marker='v', color=COLOR_VENTA, s=100,
        edgecolors='black', label='Venta', zorder=3
    )
    
    ax1.set_title(f'Bollinger Bands - {activo}', fontsize=14, pad=20)
    ax1.set_ylabel('Precio (USD)')
    ax1.legend(loc='upper left')
    ax1.grid(True, alpha=0.4)
    
    # Gráfico de volatilidad
    ax2.plot(senales['std'], label='Volatilidad', color='#9467bd', alpha=0.8)
    ax2.set_ylabel('Desviación Estándar')
    ax2.grid(True, alpha=0.4)
    
    # Formato de fechas
    locator = mdates.AutoDateLocator()
    formatter = mdates.ConciseDateFormatter(locator)
    ax1.xaxis.set_major_locator(locator)
    ax1.xaxis.set_major_formatter(formatter)
    
    plt.tight_layout()
    return fig

# Parámetros
ventana = 20
desviaciones = 2
capital_inicial = 5000

# Procesamiento para cada activo
resultados = []
for activo in datos.columns:
    precios = datos[activo]
    
    # Generar señales
    senales = generar_senales_bollinger(
        precios,
        ventana=ventana,
        desviaciones=desviaciones
    )
    
    # Ejecutar operaciones
    historial, operaciones = ejecutar_operaciones(precios, senales, capital_inicial)
    
    # Calcular métricas
    rentabilidad_bruta = historial[-1] - capital_inicial  # <- Nueva línea
    rentabilidad_porcentual = (rentabilidad_bruta / capital_inicial) * 100
    
    # Almacenar resultados (actualizado)
    resultados.append({
        'Activo': activo,
        'Capital Final': round(historial[-1], 2),
        'Rentabilidad Bruta': round(rentabilidad_bruta, 2),  # <- Añadido
        'Rentabilidad %': round(rentabilidad_porcentual, 2),
        'Operaciones': len(operaciones)
    })
    
    # Generar y guardar gráfico
    fig = graficar_bollinger(activo, precios, senales, operaciones)
    plt.savefig(f'bollinger_{activo}.png', dpi=150, bbox_inches='tight')
    plt.close()

# Mostrar resultados
resultados_df_b = pd.DataFrame(resultados)
print("\nResultados Bollinger Bands:")
print(resultados_df_b.to_string(index=False))


Resultados Bollinger Bands:
Activo  Capital Final  Rentabilidad Bruta  Rentabilidad %  Operaciones
  AAPL       12643.34             7643.34          152.87           57
  AMZN       12364.34             7364.34          147.29           45
 BRK-B       10091.06             5091.06          101.82           58
 GOOGL       18657.50            13657.50          273.15           52
   JNJ        7262.82             2262.82           45.26           50
   JPM       21388.66            16388.66          327.77           61
    MA       13737.18             8737.18          174.74           48
  META       10227.52             5227.52          104.55           43
  MSFT       18996.11            13996.11          279.92           51
  NVDA       75640.24            70640.24         1412.80           57
    PG       11424.09             6424.09          128.48           58
  TSLA       11991.80             6991.80          139.84           51
   UNH       19362.82            14362.82       

# **RSI**

In [38]:
# rsi_corregido.py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

COLOR_COMPRA = '#4B9B44'
COLOR_VENTA = '#DE4F36'

def generar_senales_rsi(precios, ventana=14, umbral_compra=30, umbral_venta=70):
    """Genera señales de trading usando RSI"""
    delta = precios.diff()
    ganancia = delta.where(delta > 0, 0)
    perdida = -delta.where(delta < 0, 0)
    
    avg_ganancia = ganancia.rolling(ventana).mean()
    avg_perdida = perdida.rolling(ventana).mean()
    
    rs = avg_ganancia / (avg_perdida + 1e-10)
    rsi = 100 - (100 / (1 + rs))
    
    senales = pd.DataFrame(index=precios.index)
    senales['rsi'] = rsi
    senales['senal'] = 0
    senales['senal'] = np.where(rsi < umbral_compra, 1, senales['senal'])
    senales['senal'] = np.where(rsi > umbral_venta, -1, senales['senal'])
    
    return senales

def ejecutar_operaciones(precios, senales, capital_inicial=5000):
    """Ejecuta las operaciones basadas en las señales"""
    capital = capital_inicial
    acciones = 0
    operaciones = []
    historial = []
    
    for fecha in precios.index:
        precio_actual = precios[fecha]
        senal = senales.loc[fecha, 'senal']
        
        if senal == 1 and capital > 0:  # Compra
            acciones = capital / precio_actual
            capital = 0
            operaciones.append(('compra', fecha, precio_actual))
            
        elif senal == -1 and acciones > 0:  # Venta
            capital = acciones * precio_actual
            acciones = 0
            operaciones.append(('venta', fecha, precio_actual))
            
        # Registrar valor del portafolio
        valor_actual = capital + (acciones * precio_actual)
        historial.append(valor_actual)
    
    return historial, operaciones  # Return corregido

def graficar_rsi(activo, precios, senales, operaciones):
    """Genera gráfico completo con RSI y operaciones"""
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), sharex=True, gridspec_kw={'height_ratios': [3, 1]})
    
    # Gráfico de precios
    ax1.plot(precios, label='Precio', color='#1f77b4', alpha=0.8, linewidth=1.5)
    
    # Marcadores de operaciones
    compras = [op for op in operaciones if op[0] == 'compra']
    ventas = [op for op in operaciones if op[0] == 'venta']
    
    ax1.scatter(
        x=[op[1] for op in compras],
        y=[op[2] for op in compras],
        marker='^', color=COLOR_COMPRA, s=100,
        edgecolors='black', label='Compra', zorder=3
    )
    
    ax1.scatter(
        x=[op[1] for op in ventas],
        y=[op[2] for op in ventas],
        marker='v', color=COLOR_VENTA, s=100,
        edgecolors='black', label='Venta', zorder=3
    )
    
    ax1.set_title(f'RSI - {activo}', fontsize=14, pad=20)
    ax1.set_ylabel('Precio (USD)')
    ax1.legend(loc='upper left')
    ax1.grid(True, alpha=0.4)
    
    # Gráfico RSI
    ax2.plot(senales['rsi'], label='RSI', color='#9467bd', alpha=0.8)
    ax2.axhline(umbral_venta, color=COLOR_VENTA, linestyle='--', alpha=0.7, label='Sobrecompra')
    ax2.axhline(umbral_compra, color=COLOR_COMPRA, linestyle='--', alpha=0.7, label='Sobreventa')
    ax2.fill_between(precios.index, umbral_compra, umbral_venta, 
                    where=senales['rsi'].between(umbral_compra, umbral_venta),
                    color='gray', alpha=0.1)
    ax2.set_ylabel('RSI')
    ax2.set_ylim(0, 100)
    ax2.grid(True, alpha=0.4)
    
    # Formato de fechas
    locator = mdates.AutoDateLocator()
    formatter = mdates.ConciseDateFormatter(locator)
    ax1.xaxis.set_major_locator(locator)
    ax1.xaxis.set_major_formatter(formatter)
    
    plt.tight_layout()
    return fig

# Parámetros
ventana_rsi = 14
umbral_compra = 30
umbral_venta = 70
capital_inicial = 5000

# Procesamiento para cada activo
resultados = []
for activo in datos.columns:
    precios = datos[activo]
    
    # Generar señales
    senales = generar_senales_rsi(
        precios,
        ventana=ventana_rsi,
        umbral_compra=umbral_compra,
        umbral_venta=umbral_venta
    )
    
    # Ejecutar operaciones
    historial, operaciones = ejecutar_operaciones(precios, senales, capital_inicial)
    
    # Calcular métricas
    rentabilidad_bruta = historial[-1] - capital_inicial  # <- Nueva línea
    rentabilidad_porcentual = (rentabilidad_bruta / capital_inicial) * 100
    
    # Almacenar resultados (actualizado)
    resultados.append({
        'Activo': activo,
        'Capital Final': round(historial[-1], 2),
        'Rentabilidad Bruta': round(rentabilidad_bruta, 2),  # <- Añadido
        'Rentabilidad %': round(rentabilidad_porcentual, 2),
        'Operaciones': len(operaciones)
    })
    
    # Generar y guardar gráfico
    fig = graficar_rsi(activo, precios, senales, operaciones)
    plt.savefig(f'rsi_{activo}.png', dpi=150, bbox_inches='tight')
    plt.close()

# Mostrar resultados
resultados_df_r = pd.DataFrame(resultados)
print("\nResultados RSI:")
print(resultados_df_r.to_string(index=False))


Resultados RSI:
Activo  Capital Final  Rentabilidad Bruta  Rentabilidad %  Operaciones
  AAPL        8345.90             3345.90           66.92           58
  AMZN        7448.36             2448.36           48.97           45
 BRK-B       13158.14             8158.14          163.16           62
 GOOGL       10794.74             5794.74          115.89           45
   JNJ       10753.80             5753.80          115.08           66
   JPM       17119.24            12119.24          242.38           59
    MA       11062.56             6062.56          121.25           42
  META        6220.49             1220.49           24.41           43
  MSFT       17935.89            12935.89          258.72           53
  NVDA       22107.48            17107.48          342.15           46
    PG        8398.98             3398.98           67.98           54
  TSLA       11716.78             6716.78          134.34           61
   UNH       20607.61            15607.61          312.15   

In [39]:
print(resultados_df_m.to_string(index=False))

Activo  Capital Final  Rentabilidad Bruta  Rentabilidad %  Operaciones
  AAPL       17419.04            12419.04          248.38           56
  AMZN       18851.60            13851.60          277.03           48
 BRK-B        8723.34             3723.34           74.47           55
 GOOGL       14518.55             9518.55          190.37           54
   JNJ        5632.89              632.89           12.66           63
   JPM       19003.64            14003.64          280.07           47
    MA       14996.27             9996.27          199.93           47
  META       50934.28            45934.28          918.69           45
  MSFT       14614.11             9614.11          192.28           54
  NVDA      230965.94           225965.94         4519.32           48
    PG        8710.01             3710.01           74.20           57
  TSLA       20937.68            15937.68          318.75           60
   UNH        7049.47             2049.47           40.99           54
     V

In [40]:
print(resultados_df_b.to_string(index=False))

Activo  Capital Final  Rentabilidad Bruta  Rentabilidad %  Operaciones
  AAPL       12643.34             7643.34          152.87           57
  AMZN       12364.34             7364.34          147.29           45
 BRK-B       10091.06             5091.06          101.82           58
 GOOGL       18657.50            13657.50          273.15           52
   JNJ        7262.82             2262.82           45.26           50
   JPM       21388.66            16388.66          327.77           61
    MA       13737.18             8737.18          174.74           48
  META       10227.52             5227.52          104.55           43
  MSFT       18996.11            13996.11          279.92           51
  NVDA       75640.24            70640.24         1412.80           57
    PG       11424.09             6424.09          128.48           58
  TSLA       11991.80             6991.80          139.84           51
   UNH       19362.82            14362.82          287.26           59
     V

In [41]:
print(resultados_df_r.to_string(index=False))

Activo  Capital Final  Rentabilidad Bruta  Rentabilidad %  Operaciones
  AAPL        8345.90             3345.90           66.92           58
  AMZN        7448.36             2448.36           48.97           45
 BRK-B       13158.14             8158.14          163.16           62
 GOOGL       10794.74             5794.74          115.89           45
   JNJ       10753.80             5753.80          115.08           66
   JPM       17119.24            12119.24          242.38           59
    MA       11062.56             6062.56          121.25           42
  META        6220.49             1220.49           24.41           43
  MSFT       17935.89            12935.89          258.72           53
  NVDA       22107.48            17107.48          342.15           46
    PG        8398.98             3398.98           67.98           54
  TSLA       11716.78             6716.78          134.34           61
   UNH       20607.61            15607.61          312.15           63
     V